Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### L2 regularization for logistic model

In [5]:
batch_size = 128
l2_loss_param = 0.001


graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + l2_loss_param * tf.nn.l2_loss(weights)

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)


In [6]:
num_steps = 3001
print_interval = 500

def run_graph(steps, batch_size, output_interval):
    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        print("Initialized")
        for step in range(steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % output_interval == 0 or step == steps - 1):
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
        
run_graph(num_steps, batch_size, print_interval)

Initialized
Minibatch loss at step 0: 20.165991
Minibatch accuracy: 11.7%
Validation accuracy: 13.5%
Minibatch loss at step 500: 3.067460
Minibatch accuracy: 71.9%
Validation accuracy: 76.8%
Minibatch loss at step 1000: 1.668937
Minibatch accuracy: 81.2%
Validation accuracy: 78.7%
Minibatch loss at step 1500: 1.289947
Minibatch accuracy: 78.9%
Validation accuracy: 77.8%
Minibatch loss at step 2000: 1.022923
Minibatch accuracy: 79.7%
Validation accuracy: 81.4%
Minibatch loss at step 2500: 0.782453
Minibatch accuracy: 83.6%
Validation accuracy: 80.4%
Minibatch loss at step 3000: 0.735727
Minibatch accuracy: 83.6%
Validation accuracy: 82.1%
Test accuracy: 89.2%


### L2 regularization for neural network model

In [7]:
hidden_layer_size = 1024


graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Variables.
  weights_first = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  biases_first = tf.Variable(tf.zeros([hidden_layer_size]))

  weights_second = tf.Variable(
    tf.truncated_normal([hidden_layer_size, num_labels]))
  biases_second = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits_first = tf.matmul(tf_train_dataset, weights_first) + biases_first
  relu_output = tf.nn.relu(logits_first)
  logits_second = tf.matmul(relu_output, weights_second) + biases_second
  loss = (tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_second))
         + l2_loss_param * tf.nn.l2_loss(weights_first)
         + l2_loss_param * tf.nn.l2_loss(weights_second))

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_second)
  valid_prediction = tf.nn.softmax(
      tf.matmul(
          tf.nn.relu(
              tf.matmul(tf_valid_dataset, weights_first) + biases_first), 
          weights_second)
      + biases_second)

  test_prediction = tf.nn.softmax(
      tf.matmul(
          tf.nn.relu(
              tf.matmul(tf_test_dataset, weights_first) + biases_first), 
          weights_second)
      + biases_second)
    
run_graph(num_steps, batch_size, print_interval)

Initialized
Minibatch loss at step 0: 719.785706
Minibatch accuracy: 7.8%
Validation accuracy: 38.0%
Minibatch loss at step 500: 199.521255
Minibatch accuracy: 81.2%
Validation accuracy: 81.3%
Minibatch loss at step 1000: 117.743958
Minibatch accuracy: 81.2%
Validation accuracy: 81.2%
Minibatch loss at step 1500: 68.589951
Minibatch accuracy: 85.2%
Validation accuracy: 82.3%
Minibatch loss at step 2000: 41.721336
Minibatch accuracy: 82.0%
Validation accuracy: 84.6%
Minibatch loss at step 2500: 25.208063
Minibatch accuracy: 86.7%
Validation accuracy: 85.8%
Minibatch loss at step 3000: 15.482451
Minibatch accuracy: 86.7%
Validation accuracy: 87.1%
Test accuracy: 93.0%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

### Overfitting: logistic regression, no L2 regularization

In [8]:
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    
def run_graph_overfit(steps, batch_size, output_interval):
    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        print("Initialized")
        for step in range(steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = batch_size * np.random.choice(range(5))
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % output_interval == 0 or step == steps - 1):
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    
run_graph_overfit(num_steps, batch_size, print_interval)

Initialized
Minibatch loss at step 0: 16.157982
Minibatch accuracy: 12.5%
Validation accuracy: 10.9%
Minibatch loss at step 500: 0.053704
Minibatch accuracy: 100.0%
Validation accuracy: 66.6%
Minibatch loss at step 1000: 0.013677
Minibatch accuracy: 100.0%
Validation accuracy: 68.1%
Minibatch loss at step 1500: 0.040238
Minibatch accuracy: 99.2%
Validation accuracy: 68.4%
Minibatch loss at step 2000: 0.031885
Minibatch accuracy: 99.2%
Validation accuracy: 68.8%
Minibatch loss at step 2500: 0.027655
Minibatch accuracy: 99.2%
Validation accuracy: 69.0%
Minibatch loss at step 3000: 0.005132
Minibatch accuracy: 100.0%
Validation accuracy: 69.5%
Test accuracy: 75.9%


### Overfitting: logistic regression, using L2 regularization shows better result than without L2 regularization

In [9]:
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + l2_loss_param * tf.nn.l2_loss(weights)

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    
def run_graph_overfit(steps, batch_size, output_interval):
    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        print("Initialized")
        for step in range(steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = batch_size * np.random.choice(range(5))
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % output_interval == 0 or step == steps - 1):
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    
run_graph_overfit(num_steps, batch_size, print_interval)

Initialized
Minibatch loss at step 0: 22.855618
Minibatch accuracy: 3.1%
Validation accuracy: 10.5%
Minibatch loss at step 500: 1.822411
Minibatch accuracy: 100.0%
Validation accuracy: 67.9%
Minibatch loss at step 1000: 1.121768
Minibatch accuracy: 100.0%
Validation accuracy: 70.2%
Minibatch loss at step 1500: 0.759718
Minibatch accuracy: 98.4%
Validation accuracy: 71.5%
Minibatch loss at step 2000: 0.463993
Minibatch accuracy: 100.0%
Validation accuracy: 72.9%
Minibatch loss at step 2500: 0.314765
Minibatch accuracy: 100.0%
Validation accuracy: 73.5%
Minibatch loss at step 3000: 0.228419
Minibatch accuracy: 100.0%
Validation accuracy: 74.1%
Test accuracy: 81.1%


### Overfitting: 1-hidden layer neural network, no L2 regularization

In [10]:
graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Variables.
  weights_first = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  biases_first = tf.Variable(tf.zeros([hidden_layer_size]))

  weights_second = tf.Variable(
    tf.truncated_normal([hidden_layer_size, num_labels]))
  biases_second = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits_first = tf.matmul(tf_train_dataset, weights_first) + biases_first
  relu_output = tf.nn.relu(logits_first)
  logits_second = tf.matmul(relu_output, weights_second) + biases_second
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_second))
         

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_second)
  valid_prediction = tf.nn.softmax(
      tf.matmul(
          tf.nn.relu(
              tf.matmul(tf_valid_dataset, weights_first) + biases_first), 
          weights_second)
      + biases_second)

  test_prediction = tf.nn.softmax(
      tf.matmul(
          tf.nn.relu(
              tf.matmul(tf_test_dataset, weights_first) + biases_first), 
          weights_second)
      + biases_second)

run_graph_overfit(num_steps, batch_size, print_interval)

Initialized
Minibatch loss at step 0: 400.077393
Minibatch accuracy: 8.6%
Validation accuracy: 23.8%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 76.3%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 76.8%
Minibatch loss at step 1500: 0.000002
Minibatch accuracy: 100.0%
Validation accuracy: 77.0%
Minibatch loss at step 2000: 0.029012
Minibatch accuracy: 99.2%
Validation accuracy: 76.9%
Minibatch loss at step 2500: 0.003157
Minibatch accuracy: 100.0%
Validation accuracy: 76.8%
Minibatch loss at step 3000: 0.000003
Minibatch accuracy: 100.0%
Validation accuracy: 76.8%
Test accuracy: 83.2%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [13]:
keep_prob = 0.5

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Variables.
  weights_first = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  biases_first = tf.Variable(tf.zeros([hidden_layer_size]))

  weights_second = tf.Variable(
    tf.truncated_normal([hidden_layer_size, num_labels]))
  biases_second = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits_first = tf.matmul(tf_train_dataset, weights_first) + biases_first
  relu_output = tf.nn.relu(logits_first)
  dropout_output = tf.nn.dropout(relu_output, keep_prob)

  logits_second = tf.matmul(dropout_output, weights_second) + biases_second
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_second))
       

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_second)
  valid_prediction = tf.nn.softmax(
      tf.matmul(
          tf.nn.relu(
              tf.matmul(tf_valid_dataset, weights_first) + biases_first), 
          weights_second)
      + biases_second)

  test_prediction = tf.nn.softmax(
      tf.matmul(
          tf.nn.relu(
              tf.matmul(tf_test_dataset, weights_first) + biases_first), 
          weights_second)
      + biases_second)
    
run_graph(num_steps, batch_size, print_interval)

Initialized
Minibatch loss at step 0: 473.566345
Minibatch accuracy: 9.4%
Validation accuracy: 27.0%
Minibatch loss at step 500: 22.055038
Minibatch accuracy: 77.3%
Validation accuracy: 79.6%
Minibatch loss at step 1000: 32.207401
Minibatch accuracy: 68.0%
Validation accuracy: 79.3%
Minibatch loss at step 1500: 8.705660
Minibatch accuracy: 75.8%
Validation accuracy: 79.3%
Minibatch loss at step 2000: 27.229733
Minibatch accuracy: 65.6%
Validation accuracy: 78.1%
Minibatch loss at step 2500: 17.661911
Minibatch accuracy: 70.3%
Validation accuracy: 80.3%
Minibatch loss at step 3000: 5.553663
Minibatch accuracy: 66.4%
Validation accuracy: 79.6%
Test accuracy: 86.3%


### Use dropout on overfitting case

In [14]:
graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Variables.
  weights_first = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  biases_first = tf.Variable(tf.zeros([hidden_layer_size]))

  weights_second = tf.Variable(
    tf.truncated_normal([hidden_layer_size, num_labels]))
  biases_second = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits_first = tf.matmul(tf_train_dataset, weights_first) + biases_first
  relu_output = tf.nn.relu(logits_first)
  dropout_output = tf.nn.dropout(relu_output, keep_prob)

  logits_second = tf.matmul(dropout_output, weights_second) + biases_second
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_second))
       

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_second)
  valid_prediction = tf.nn.softmax(
      tf.matmul(
          tf.nn.relu(
              tf.matmul(tf_valid_dataset, weights_first) + biases_first), 
          weights_second)
      + biases_second)

  test_prediction = tf.nn.softmax(
      tf.matmul(
          tf.nn.relu(
              tf.matmul(tf_test_dataset, weights_first) + biases_first), 
          weights_second)
      + biases_second)
    
run_graph_overfit(num_steps, batch_size, print_interval)

Initialized
Minibatch loss at step 0: 528.977600
Minibatch accuracy: 2.3%
Validation accuracy: 25.6%
Minibatch loss at step 500: 12.136149
Minibatch accuracy: 97.7%
Validation accuracy: 76.1%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 76.7%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 78.3%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 78.2%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.8%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 78.0%
Test accuracy: 84.9%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [130]:
hidden_layer_size = 1024
starter_learning_rate = 0.001
l2_loss_param = 0.01
keep_prob = 0.5
weight_stddev = 0.5

def calc_single_hidden_layer(x, weight, bias, dropout_keep_prob = 1):
    logits = tf.matmul(x, weight) + bias
    relu_output = tf.nn.relu(logits)
    dropout_output = tf.nn.dropout(relu_output, dropout_keep_prob)
    return dropout_output

def calc_single_output_layer(x, weight, bias):
    matmul_result = tf.check_numerics(tf.matmul(x, weight), "nan or inf of tf.matmul(x, weight)")
    logits = matmul_result + bias
    #logits = tf.matmul(x, weight) + bias
    return logits

def calc_multi_layer(x, weights, biases, dropout_keep_prob = 1):
    dropout_output_first = calc_single_hidden_layer(x, weights['h1'], biases['h1'], dropout_keep_prob)
    dropout_output_second = calc_single_hidden_layer(dropout_output_first, weights['h2'], biases['h2'], dropout_keep_prob)
    dropout_output_third = calc_single_hidden_layer(dropout_output_second, weights['h3'], biases['h3'], dropout_keep_prob)
    output_third = calc_single_output_layer(dropout_output_third, weights['out'], biases['out'])
    return output_third

graph = tf.Graph()
with graph.as_default():
    
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = {'h1':tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size], stddev = weight_stddev)),
             'h2':tf.Variable(tf.truncated_normal([hidden_layer_size, hidden_layer_size], stddev = weight_stddev)),
             'h3':tf.Variable(tf.truncated_normal([hidden_layer_size, hidden_layer_size], stddev = weight_stddev)),
             'out':tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels], stddev = weight_stddev))}

    biases = {'h1':tf.Variable(tf.zeros([hidden_layer_size])),
            'h2':tf.Variable(tf.zeros([hidden_layer_size])),
            'h3':tf.Variable(tf.zeros([hidden_layer_size])),
            'out':tf.Variable(tf.zeros([num_labels]))}
    
    global_step = tf.Variable(0, trainable=False)

    # Training computation.
    logits_final_layer = calc_multi_layer(tf_train_dataset, weights, biases, keep_prob)

    loss = (tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_final_layer))
            + l2_loss_param * tf.nn.l2_loss(weights['h1'])
            + l2_loss_param * tf.nn.l2_loss(weights['h2'])
            + l2_loss_param * tf.nn.l2_loss(weights['h3'])
            + l2_loss_param * tf.nn.l2_loss(weights['out']))

    # Optimizer.
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           10, 0.95, staircase=True)

    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits_final_layer)
    valid_prediction = tf.nn.softmax(calc_multi_layer(tf_valid_dataset, weights, biases))
    test_prediction = tf.nn.softmax(calc_multi_layer(tf_test_dataset, weights, biases))
    
    check_op = tf.add_check_numerics_ops()
    
# Troubleshooting nan or inf issue steps:
# 1. Use tf.add_check_numerics_ops() to find out where generates the first nan or inf value
# 2. Use tf.check_numerics to confirm/find out the exact code location where the first nan or inf generated according to the result of step 1
# 3. Print out the related values in session.run(...) to find out the value pattern of each step: the value got bigger and bigger eventually overflow happend
def run_graph(steps, batch_size, output_interval):
    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        print("Initialized")
        for step in range(steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}

            #print(step)
#             print(session.run(logits_final_layer, feed_dict=feed_dict))
#             _, l, predictions, op = session.run(
#                [optimizer, loss, train_prediction, check_op], feed_dict=feed_dict)
            _, l, predictions = session.run(
               [optimizer, loss, train_prediction], feed_dict=feed_dict)
            
            if (step % output_interval == 0 or step == steps - 1):
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

run_graph(num_steps, batch_size, print_interval)

Initialized
Minibatch loss at step 0: 29093.755859
Minibatch accuracy: 11.7%
Validation accuracy: 21.4%
Minibatch loss at step 500: 3832.422363
Minibatch accuracy: 65.6%
Validation accuracy: 80.9%
Minibatch loss at step 1000: 3800.354736
Minibatch accuracy: 64.8%
Validation accuracy: 81.0%
Minibatch loss at step 1500: 4128.966309
Minibatch accuracy: 60.2%
Validation accuracy: 81.0%
Minibatch loss at step 2000: 3995.644531
Minibatch accuracy: 59.4%
Validation accuracy: 81.0%
Minibatch loss at step 2500: 3801.587158
Minibatch accuracy: 69.5%
Validation accuracy: 81.0%
Minibatch loss at step 3000: 4106.350586
Minibatch accuracy: 63.3%
Validation accuracy: 81.0%
Test accuracy: 87.4%
